In [5]:
import tensortools as tt
import numpy as np
import matplotlib.pyplot as plt

import musclebeachtools
from braingeneers import neuron
import glob
import sys

from scipy.ndimage import gaussian_filter1d

In [6]:
sys.modules['musclebeachtools_hlab'] = sys.modules['musclebeachtools']
sys.modules['musclebeachtools_hlab.musclebeachtools'] = sys.modules['musclebeachtools']

In [8]:
def get_well(chs):
    '''chs -- file list in the results folder of the axion data output'''
    # Sort
    chs = sorted(chs)

    # Remove rasters 
    chs = [ch for ch in chs if 'rasters' not in ch]

    well_dict = {}
    seen_ch = []

    for ch in chs:

        well_grp = ch.split('/')[-1]
#         print(well_grp)
        well, grp = well_grp.split('chgroup')
#         print(well,grp)

        #Maps group to full name
        temp = {well+grp:ch}

        if well not in seen_ch:
            seen_ch.append(well)
            well_dict[well] = {}

        well_dict[well][int(grp)] = ch


    return well_dict

## Load in spike times from experiment

In [ ]:
#List possible experiments
chs = glob.glob("/Users/seymour/research/braingeneers/ephys/root/data/2020-12-01-e-MGK-primary-axion-stim-500mV-20sec/results/*")
well_dict = get_well(chs)

nf = glob.glob(well_dict['A1'][1] + '/spikeintf/outputs/neurons*')
data = np.load(nf[0],allow_pickle=True)

